### Python Preamble

In [2]:
# Disulfide Bond Analysis
# Author: Eric G. Suchanek, PhD.
# Last revision: 12/16/22 -egs-
# Cα Cβ Sγ

%matplotlib inline

import matplotlib.pyplot as plt
import plotly_express as px

import proteusPy
from proteusPy import *

from proteusPy.proteusGlobals import *

import pandas as pd

# the locations below represent the actual location on the dev drive.
# location for PDB repository
PDB_BASE = '/Users/egs/PDB/'

# location of cleaned PDB files
PDB = '/Users/egs/PDB/good/'

# location of the compressed Disulfide .pkl files
MODELS = f'{PDB_BASE}models/'


# Analysis of Disulfide Bonds in Proteins Within the RCSB Protein Data Bank
*Eric G. Suchanek, PhD. (suchanek@mac.com)* <br> <br>

## Summary
I describe the results of a structural analysis of Disulfide bonds contained in 36,362 proteins within the RCSB Protein databank, https://www.rcsb.org. These protein structures contained 294,478 Disulfide Bonds.  The analysis utilizes Python functions from my ``ProteusPy`` package https://github.com/suchanek/proteusPy/, which is built upon the excellent ``BioPython`` library (https://www.biopython.org). 

This work represents a reprise of my original Disulfide modeling analysis conducted in 1986 ([publications](#publications) item 1) as part of my dissertation. Given the original Disulfide database contained only 2xx Disulfide Bonds I felt it would be interesting to revisit the RCSB and mine the thousands of new structures. The initial results are described in the cells below.

### Requirements
 - Biopython patched version, or my delta applied
 - proteusPy: https://github.com/suchanek/proteusPy/

## Introduction
Disulfide bonds are important covalent stabilizing elements in proteins. They are formed when two Sulphur-containing Cysteine (Cys) amino acid residues are close enough and in the correct geometry to form a S-S covalent bond with their terminal sidechain Sγ atoms. Disulfide bonds most commonly occur between alpha helices and greatly enhance a protein's stability to denaturation. 


## Download PDB Files containing Disulfide Bonds

The RCSB query yielded 

In [ ]:

# DownloadDisulfides(pdb_home=PDB_BASE, model_home=MODELS, reset=False)


## Extract the Disulfides from the PDB files
The function ``ExtractDisulfides()`` processes all the .ent files in ``PDB_DIR`` and creates two .pkl files representing the Disulfide bonds contained in the scanned directory. In addition, a .csv file containing problem IDs is written if any are found. The .pkl files are consumed by the ``DisulfideLoader`` class and are considered private. You'll see numerous warnings during the scan. Files that are unparsable are removed and their IDs are logged to the problem_id.csv file. The default file locations are stored in the file globals.py and are the used by ``DisulfideExtractor()`` in the absence of arguments passed. The Disulfide parser is very stringent and will reject disulfide bonds with missing atoms or disordered atoms.


Outputs are saved in ``MODEL_DIR``:
1) ``SS_PICKLE_FILE``: The ``DisulfideList`` of ``Disulfide`` objects initialized from the PDB file scan, needed by the ``DisulfideLoader()`` class.
2) ``SS_DICT_PICKLE_FILE``: the ``Dict Disulfide`` objects also needed by the ``DisulfideLoader()`` class
3) ``PROBLEM_ID_FILE``: a .csv containining the problem ids.

In general, the process only needs to be run once for a full scan. Setting the ``numb`` argument to -1 scans the entire directory. Entering a positive number allows parsing a subset of the dataset, which is useful when debugging. Setting ``verbose`` enables verbose messages. Setting ``quiet`` to ``True`` disables all warnings.

NB: A extraction of the initial disulfide bond-containing files (> 36000 files) takes about 1.25 hours on a 2020 MacbookPro with M1 Pro chip, 16GB RAM, 1TB SSD. The resulting .pkl files consume approximately 1GB of disk space, and equivalent RAM used when loaded.

In [7]:


ExtractDisulfides(numb=1000, pdbdir=PDB, modeldir=MODELS, verbose=False, quiet=True)


100%|███████████████████████████████████████████| 1000/1000 [01:37<00:00, 10.26it/s, ID=3fyi, Bad=0]


Saving 8128 Disulfides to file: /Users/egs/PDB/models/PDB_all_ss.pkl
Saving 1000 Disulfide-containing PDB IDs to file: /Users/egs/PDB/models/PDB_all_ss_dict.pkl
Saving torsions to file: /Users/egs/PDB/models/PDB_SS_torsions.csv
Disulfide Extraction complete! Elapsed time: 0:01:38.885941 (h:m:s)


## Load the Disulfide Data
Now that the Disulfides have been extracted and the Disulfide .pkl files have been created we can load them into memory using the DisulfideLoader() class. This class stores the Disulfides internally as a DisulfideList and a dict. Array indexing operations including slicing have been overloaded, enabling straightforward access to the Disulfide bonds, both in aggregate and by residue. After loading the .pkl files the Class creates a Pandas ``DataFrame`` object consisting of the disulfide ID, all sidechain dihedral angles, and the computed Disulfide bond torsional energy.

NB: Loading the data takes 2.5 minutes on my MacbookPro. Be patient if it seems to take a long time to load.

In [8]:
# when running from the repo the local copy of the Disulfides is in ../pdb/models
# PDB_BASE = '../pdb/'

# location of the compressed Disulfide .pkl files
# MODELS = f'{PDB_BASE}models/'

PDB_SS = None
PDB_SS = DisulfideLoader(verbose=True, modeldir=MODELS)


Reading disulfides from: /Users/egs/PDB/models/PDB_all_ss.pkl
Disulfides Read: 8128
Reading disulfide dict from: /Users/egs/PDB/models/PDB_all_ss_dict.pkl
Reading Torsion DF /Users/egs/PDB/models/PDB_SS_torsions.csv.
Read torsions DF.
PDB IDs parsed: 1000
Total Space Used: 1761893 bytes.


## Examine the Disulfide Torsions
The disulfide bond's overall conformation is defined by the sidechain dihedral angles $\chi_{1}$-$\chi_{5}$. These values along with the approximate torsional energy are stored in the DisulfideLoader() class. We access them via the ``DisulfideList.getTorsions()`` function.

We can get a quick look at their overall statistics using the ``Pandas.describe()`` function.


In [10]:
#SS_df = pd.DataFrame()

_SSdf = PDB_SS.getTorsions()                     # retrieve the torsions dataframe
cols = ['chi1', 'chi2', 'chi3', 'chi4', 'chi5', 'energy']
SS_df = _SSdf[cols].copy()

print(SS_df.head(5))
SS_df[cols].describe()

         chi1        chi2       chi3       chi4        chi5    energy
0  174.629233   82.517710 -83.322249 -62.523644  -73.827286  1.696237
1  -50.239063  -85.583916  97.275447  70.535692  179.046592  2.112566
2   62.598713  172.940042 -95.352637 -23.070934  -55.158480  2.331733
3  173.666078   88.297996 -82.387276 -65.997032  -72.289506  1.958823
4  -56.410909  -81.401941  94.310784  67.035993  178.852441  1.444608


,chi1,chi2,chi3,chi4,chi5,energy
count,8128.000000,8128.000000,8128.000000,8128.000000,8128.000000,8128.000000
mean,-46.404195,-4.646353,-4.989255,-25.741060,-28.281897,3.736626
std,103.652993,110.274206,94.248718,112.041293,99.673350,2.357644
min,-179.990815,-180.000000,-180.000000,-180.000000,-179.999305,0.494753
25%,-90.475100,-86.930099,-88.665481,-99.478328,-74.038973,2.063532
50%,-64.151503,-54.482183,-65.612669,-68.762631,-59.427880,3.291202
75%,-41.536990,100.479534,93.380396,83.672793,57.571497,4.632302
max,179.945970,179.987671,179.976934,179.977181,179.990016,17.289549


## Examining the Torsion Angles



In [6]:
labels = {'value': 'Angle', 'variable': 'Torsion'}
cols = ['chi1', 'chi5']
data = SS_df[cols].copy()
data.set_index('chi1')

px.histogram(SS_df, x='chi1', y='chi5', labels=labels, histfunc='avg', nbins=90)
#px.histogram(SS_df[cols], labels=labels)


In [ ]:
px.histogram(SS_df, x='chi2', y='chi4', labels=labels, histfunc='avg', nbins=90)

We can use a hexbin plot to generate the equivalent of a heatmap to explore the symmetry for the Chi1-Chi5 and Chi2-Chi4 dihedral angles. Since the disulfide bond is symmetric about Chi3 (the S-S bond), one would expect the distribution of Chi1 to be similar to Chi5, and the same with Chi2 and Chi4.

In [ ]:

# Set the figure sizes and axis limits.
DPI = 254
WIDTH = 6.0
HEIGHT = 3.0
TORMIN = -180.0
TORMAX = 180.
GRIDSIZE = 40


In [ ]:
x = SS_df['chi1']
y = SS_df['chi5']

fig = plt.figure(figsize=(WIDTH, HEIGHT), dpi=DPI)
ax1 = plt.subplot(111)

plt.hexbin(x, y, gridsize=GRIDSIZE, cmap='nipy_spectral')
ax1.set_xlim(TORMIN, TORMAX)
ax1.set_ylim(TORMIN, TORMAX)
ax1.set_xlabel('Chi1')
ax1.set_ylabel('Chi5')
ax1.set_title('Chi1-Chi5 Torsion Angle Symmetry')
ax1.grid()

plt.show()


In [ ]:
#

fig = plt.figure(figsize=(WIDTH, HEIGHT), dpi=DPI)
ax1 = plt.subplot(111)

x = SS_df['chi2']
y = SS_df['chi4']

plt.hexbin(x, y, gridsize=GRIDSIZE, cmap='nipy_spectral')
ax1.set_xlim(TORMIN, TORMAX)
ax1.set_ylim(TORMIN, TORMAX)
ax1.set_xlabel('Chi2')
ax1.set_ylabel('Chi4')
ax1.set_title('Chi2-Chi4 Torsion Angle Symmetry')
ax1.grid()
plt.show()


In [ ]:
#

fig = plt.figure(figsize=(WIDTH, HEIGHT), dpi=DPI)
ax1 = plt.subplot(111)

x = SS_df['chi3']
y = SS_df['energy']

ymax = 10.0
plt.hexbin(x, y, gridsize=GRIDSIZE, cmap='nipy_spectral')
ax1.set_xlim(TORMIN, TORMAX)
ax1.set_ylim(y.min(), ymax)
ax1.set_xlabel('Chi3')
ax1.set_ylabel('Energy (kcal/mol)')
ax1.set_title('Energy vs Chi3 (S-S)')
ax1.grid()
plt.show()


In [ ]:
from sklearn.mixture import GaussianMixture
n_clusters = 3

tor_df = SS_df[['chi1', 'chi2', 'chi3', 'chi4', 'chi5', 'energy']].copy()
tor_df.head(1)
gmm_model = GaussianMixture(n_components=n_clusters)
gmm_model.fit(tor_df)



In [ ]:
cluster_labels = gmm_model.predict(tor_df)
X = pd.DataFrame(tor_df)
X['cluster'] = cluster_labels

In [ ]:
for k in range(n_clusters):
    data = X[X['cluster'] == k]
    plt.scatter(data['chi1'], data['energy'], s=2)
    

In [ ]:
from sklearn.cluster import SpectralClustering
n_clusters = 4

tor_df = SS_df[['chi1', 'chi2', 'chi3', 'chi4', 'chi5', 'energy']].copy()
X = tor_df[:5000].copy()

tor_df.head(1)
scm_model = SpectralClustering(n_clusters=n_clusters, random_state=25,
                                n_neighbors=8, affinity='nearest_neighbors')
X['cluster'] = scm_model.fit_predict(X[['chi3', 'energy']])


In [ ]:
import seaborn as sns

fig, ax = plt.subplots()
ax.set(title='Spectral Clustering')
sns.scatterplot(x='chi3', y='energy', data=X, hue='cluster', ax=ax)



In [ ]:
#
# stolen from Bert_ts_split_colab.ipynb
#

from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np


# since the predictions are logits we have to convert to the correct
# label (0 1 2)

'''
preds = []
for logit in predictions:
  lab = np.argmax(logit)
  preds.append(lab)

preds = np.array(preds)
matrix = confusion_matrix(correct_classes, preds)

conf_matrix = pd.DataFrame(matrix, index = ['Detractor','Passive','Promoter'],columns = ['Detractor','Passive','Promoter'])
#Normalizing
conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
plt.figure(figsize = (10,10))

sns.heatmap(conf_matrix, annot=True, annot_kws={"size": 10})
'''


matrix = confusion_matrix()


In [ ]:
labels = {'value': 'Chi3', 'variable': 'Variable'}

x = SS_df['chi3']
y = SS_df['energy']
df = SS_df[['chi3', 'energy']].copy()

energy_hist = numpy.histogram2d(x, y=y, bins=100)
energy_hist = energy_hist

fig = px.histogram(df, labels=labels)

fig.show()

In [ ]:
radius = y
energy = chi1_energy[1]
theta1 = chi1_hist[1]
theta2 = chi1_energy[1]


print(radius)


'''
fig = px.line_polar(energy, r=radius, theta=theta, line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",)
fig.show()
'''


In [ ]:
import plotly.express as px
df = px.data.wind()
print(df.head(2))
fig = px.scatter_polar(df, r="frequency", theta="direction",
                       color="strength", symbol="strength", size="frequency",
                       color_discrete_sequence=px.colors.sequential.Plasma_r)
fig.show()

In [ ]:
fig = px.line_polar(SS_df[['chi1']], r=SS_df[['chi1']].sum(), theta="chi1", line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",)
fig.show()

In [ ]:
cols = ['chi1', 'chi5']
px.histogram(SS_df[cols], labels=labels)

In [ ]:
cols = ['chi2', 'chi4']
px.histogram(SS_df[cols], labels=labels)

In [ ]:
px.histogram(SS_df['chi3'], labels=labels)

## Summary
Conformational analysis of 294,222 disulfide bonds in 36,362 proteins contained in the RCSB confirms the predominant conformational classes first described in my initial analysis:
- Left-Handed Spiral
- Right-Handed Hook
- Left-Handed Spiral
  

## Publications
* https://doi.org/10.1021/bi00368a023
* https://doi.org/10.1021/bi00368a024
* https://doi.org/10.1016/0092-8674(92)90140-8
* http://dx.doi.org/10.2174/092986708783330566

In [ ]:
from pyvista import set_plot_theme
set_plot_theme('document')

In [ ]:
import numpy as np

import pyvista as pv
pv.set_jupyter_backend('ipyvtklink')


def _cell_bounds(points, bound_position=0.5):
    """
    Calculate coordinate cell boundaries.

    Parameters
    ----------
    points: numpy.ndarray
        One-dimensional array of uniformly spaced values of shape (M,).

    bound_position: bool, optional
        The desired position of the bounds relative to the position
        of the points.

    Returns
    -------
    bounds: numpy.ndarray
        Array of shape (M+1,)

    Examples
    --------
    >>> a = np.arange(-1, 2.5, 0.5)
    >>> a
    array([-1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ])
    >>> cell_bounds(a)
    array([-1.25, -0.75, -0.25,  0.25,  0.75,  1.25,  1.75,  2.25])
    """
    if points.ndim != 1:
        raise ValueError("Only 1D points are allowed.")
    diffs = np.diff(points)
    delta = diffs[0] * bound_position
    bounds = np.concatenate([[points[0] - delta], points + delta])
    return bounds


# First, create some dummy data

# Approximate radius of the Earth
RADIUS = 6371.0

# Longitudes and latitudes
x = np.arange(0, 360, 5)
y = np.arange(-90, 91, 5)
y_polar = 90.0 - y  # grid_from_sph_coords() expects polar angle

xx, yy = np.meshgrid(x, y)


# x- and y-components of the wind vector
u_vec = np.cos(np.radians(xx))  # zonal
v_vec = np.sin(np.radians(yy))  # meridional

# Scalar data
scalar = u_vec**2 + v_vec**2

# Create arrays of grid cell boundaries, which have shape of (x.shape[0] + 1)
xx_bounds = _cell_bounds(x)
yy_bounds = _cell_bounds(y_polar)
# Vertical levels
# in this case a single level slightly above the surface of a sphere
levels = [RADIUS * 1.01]

In [ ]:
grid_scalar = pv.grid_from_sph_coords(xx_bounds, yy_bounds, levels)

# And fill its cell arrays with the scalar data
grid_scalar.cell_data["example"] = np.array(scalar).swapaxes(-2, -1).ravel("C")

# Make a plot
p = pv.Plotter()
p.add_mesh(pv.Sphere(radius=RADIUS))
p.add_mesh(grid_scalar, clim=[0.1, 2.0], opacity=0.5, cmap="plasma")
p.show()